In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/money-breed-dataset/monkey.csv
/kaggle/input/money-breed-dataset/Screenshot 2021-08-28 004425.png
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n816.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n810.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n8016.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n809.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n801.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n806.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n804.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n811.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation/n8/n815.jpg
/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/

In [2]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet',
                 include_top = False,
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False

# Let's print our layers
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

2024-03-09 13:13:47.473730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 13:13:47.473874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 13:13:47.619933: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


17225924/17225924 [==============================] - 1s 0us/step
0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization F

In [3]:
def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 10

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)


In [5]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128   

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/train'
validation_data_dir = '/kaggle/input/money-breed-dataset/4.1 monkey_breed.zip/monkey_breed/validation'

# Let's use some data augmentaiton
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 16

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [7]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("MnkBreed.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0.001,
                          patience = 7,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1098
nb_validation_samples =272

# We only train 5 EPOCHS
epochs = 10
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

/tmp/ipykernel_18/2121015209.py:33: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
68/68 [==============================] - ETA: 0s - loss: 2.0262 - accuracy: 0.4732
Epoch 1: val_loss improved from inf to 0.34398, saving model to MnkBreed.h5
68/68 [==============================] - 63s 889ms/step - loss: 2.0262 - accuracy: 0.4732 - val_loss: 0.3440 - val_accuracy: 0.8934


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
68/68 [==============================] - ETA: 0s - loss: 0.5738 - accuracy: 0.8179
Epoch 2: val_loss improved from 0.34398 to 0.17569, saving model to MnkBreed.h5
68/68 [==============================] - 52s 765ms/step - loss: 0.5738 - accuracy: 0.8179 - val_loss: 0.1757 - val_accuracy: 0.9412
Epoch 3/10
68/68 [==============================] - ETA: 0s - loss: 0.4381 - accuracy: 0.8678
Epoch 3: val_loss did not improve from 0.17569
68/68 [==============================] - 52s 766ms/step - loss: 0.4381 - accuracy: 0.8678 - val_loss: 0.1888 - val_accuracy: 0.9338
Epoch 4/10
68/68 [==============================] - ETA: 0s - loss: 0.4249 - accuracy: 0.8946
Epoch 4: val_loss did not improve from 0.17569
68/68 [==============================] - 53s 768ms/step - loss: 0.4249 - accuracy: 0.8946 - val_loss: 0.2290 - val_accuracy: 0.9265
Epoch 5/10
68/68 [==============================] - ETA: 0s - loss: 0.3307 - accuracy: 0.9020
Epoch 5: val_loss did not improve from 0.17569
68/68 [